https://www.youtube.com/watch?v=Wqmtf9SA_kk

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

In [ ]:
data = pd.read_csv("nybolig_data.csv")
display(data.head())
display(data.info())

In [ ]:
data

In [ ]:
data.info()

In [ ]:
from sklearn.model_selection import train_test_split
X = data.drop(['price'], axis = 1)
y = data['price']

In [ ]:
data.hist(figsize = (15, 8))

In [ ]:
plt.figure(figsize = (15, 8))
sns.heatmap(data.select_dtypes(include = np.number).corr(), annot = True, cmap = "YlGnBu")

# Preprocessing

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

In [ ]:
def preprocess_data(data, transformations: bool = False, encoding: str = 'normal', drop_low_corr: bool = False):
    # Fill the missing values
    data['year_rebuilt'] = data['year_rebuilt'].where(~data['year_rebuilt'].isna(), data['year_built']).astype(int)
    data['basement_size'] = data["basement_size"].fillna(0)
    # Drop the columns that are not needed
    data = data.drop(['url', 'address'], axis = 1)

    # Apply the transformations
    if transformations:
        data['postal_code'] = (data['postal_code'] + 1) ** 2
        data['rooms'] = np.log(data['rooms'] + 1)
        data['size'] = np.log(data['size'] + 1)
        data['basement_size'] = np.log(data['basement_size'] + 1)
        data['year_built'] = (data['year_built'] + 1) ** 2
        data['year_rebuilt'] = (data['year_rebuilt'] + 1) ** 2

    #if scaling: 
    #   scaler = StandardScaler()
    #   data = pd.DataFrame(scaler.fit_transform(data), columns = x_data.columns)

    # Encode the categorical variables
    if encoding == 'normal':
        data['type'] = data['type'].astype('category').cat.codes
        data['energy_label'] = data['energy_label'].astype('category').cat.codes
        #Another way of doing it 
        #data['type'] = data().LabelEncoder().fit_transform(data['type'])
        #data['energy_label'] = data().LabelEncoder.fit_transform(data['energy_label'])
    elif encoding == 'onehot':
        data = pd.get_dummies(data, columns = ['type', 'energy_label'], drop_first = True, dtype=int)
    else:
        raise ValueError("The encoding parameter must be either 'normal' or 'onehot'")

    # Drop the columns that have low correlation with the target variable
    if drop_low_corr:
        for column in data.select_dtypes(include = np.number).columns:
            if abs(data[column].corr(data['price'])) < 0.1:
                data = data.drop(column, axis = 1)
        
    return data

In [ ]:
preprocessed_data = preprocess_data(data, transformations = False, encoding = 'onehot', drop_low_corr = True)

In [ ]:
preprocessed_data.hist(figsize = (15, 8))

In [ ]:
plt.figure(figsize = (15, 8))
sns.heatmap(preprocessed_data.select_dtypes(include = np.number).corr(), annot = True, cmap = "YlGnBu")

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(preprocessed_data.drop(['price'], axis = 1), preprocessed_data['price'], test_size = 0.2, random_state = 0)
display(X_train)
display(y_train)


# Regression 

## Lasso and Ridge Regression 

In [ ]:
from sklearn import linear_model
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
warnings.simplefilter(action='ignore', category=FutureWarning)
np.set_printoptions(suppress=True)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def regression(model, x_train, y_train, x_test, y_test):
    model.fit(x_train, y_train)
    score = cross_val_score(model, x_train, y_train, cv=5)
    print("Scores", score)
    print("Test score: ", model.score(x_test, y_test))
    print("\nCoefficients: ")
    for feature, coef in zip(x_train.columns, model.coef_):
        print(f"{feature}: {coef}")
    print("\nIntercept: ", model.intercept_)

    # Predicting the test set results
    y_pred = model.predict(x_test)
    
    #Evaluating the model
    print("\nEvaluation")
    print("Mean squared error: ", mean_squared_error(y_test, y_pred))
    print("Mean absolute error: ", mean_absolute_error(y_test, y_pred))
    print("R2 score: ", r2_score(y_test, y_pred))


    #Plotting the test set results
    plt.scatter(y_test, y_pred)

    # Calculate residuals
    residuals = y_pred - y_test

    # Calculate distances from the perfect fit line
    distances = np.abs(y_test - y_pred)

    # Define color gradient based on distances
    colors = distances / np.max(distances)  # Normalize distances to range [0, 1]
    # colors = plt.cm.RdYlGn_r(colors)  # Reverse the colormap: green (furthest), red (closest)

    # Plot true values vs predictions with color gradient
    plt.scatter(y_test, y_pred, c=colors)
    plt.xlabel('True values')
    plt.ylabel('Predictions')
    # Plot the perfect fit line
    plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], c='r')
    # Name the perfect fit line
    plt.legend(['Test values', 'Perfect fit'])
    plt.title(f'True values vs Predictions ({model.__class__.__name__})')
    plt.colorbar(label='Distance from Diagonal')
    plt.show()

    # Plot residuals
    plt.scatter(y_pred, residuals, c=colors)
    plt.hlines(y=0, xmin=y_test.min(), xmax=y_test.max(), colors='r')
    plt.title(f'Residual plot ({model.__class__.__name__})')
    plt.xlabel('Predicted values')
    plt.ylabel('Residuals')
    plt.colorbar(label='Distance from Diagonal')
    plt.legend(['Residuals', 'Perfect fit'])
    plt.show()


#linear_model_ = linear_model.LinearRegression()
lasso_model = linear_model.Lasso(alpha = 10)
ridge_model = linear_model.Ridge(alpha = 10)
regression(ridge_model, X_train, y_train, X_test, y_test)
regression(lasso_model, X_train, y_train, X_test, y_test)

For this results, we have the following: 
  1. Scores: These are cross-validation scores obtained using 5-fold cross-validation. They represent the R-squared values achieved by the Lasso model on different folds of the training data. Each score corresponds to one fold.
  2. Test score: This is the R-squared score of the model on the held-out test set.
  3. Coefficients: These are the weights assigned to each feature by the Lasso model. 
  4. Intercept: This is the bias term of the model.

# Extreme Gradient Boosting 

In [ ]:
import xgboost as xgb

In [ ]:
#https://xgboost.readthedocs.io/en/stable/tutorials/model.html

xgb_model = xgb.XGBRegressor(objective = 'reg:squarederror', n_estimators = 1000, learning_rate = 0.1, max_depth = 3)
xgb_model.fit(X_train, y_train)
pred = xgb_model.predict(X_test)
print("R2 score: ", r2_score(y_test, pred))
print("Mean squared error: ", mean_squared_error(y_test, pred))
print("Mean absolute error: ", mean_absolute_error(y_test, pred))
#Plot the true values vs the predicted values
plt.scatter(y_test, pred)
plt.xlabel('True values')
plt.ylabel('Predictions')
plt.title('True values vs Predictions (XGBoost)')
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], c='r')
plt.legend(['Test values', 'Perfect fit'])
plt.show()

#Residual plot
residuals = pred - y_test
plt.scatter(pred, residuals)
plt.hlines(y=0, xmin=y_test.min(), xmax=y_test.max(), colors='r')
plt.title('Residual plot (XGBoost)')
plt.xlabel('Predicted values')
plt.ylabel('Residuals')
plt.legend(['Residuals', 'Perfect fit'])

# Neural Network Apporach 

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [ ]:
def neural_network(x_train, y_train, x_test, y_test): 
  model = Sequential()

  #Adding layers 
  num_features = x_train.shape[1]
  model.add(Dense(64, input_dim = num_features, activation = 'relu'))
  model.add(Dense(32, activation = 'relu'))
  model.add(Dense(16, activation = 'relu'))
  model.add(Dense(8, activation = 'relu'))
  model.add(Dense(1, activation = 'linear'))

  #Compiling and fitting 
  model.compile(optimizer = 'adam', loss = 'mean_squared_error')
  model.fit(x_train, y_train, epochs = 400, batch_size = 32, verbose = 0)
  print("Test score: ", model.evaluate(x_test, y_test))

  # Predicting the test set results
  y_pred = model.predict(x_test).flatten()

  #Evaluating the model
  print("\nEvaluation")
  print("R2 score: ", r2_score(y_test, y_pred))
  print("Mean squared error: ", mean_squared_error(y_test, y_pred))
  print("Mean absolute error: ", mean_absolute_error(y_test, y_pred))

  #Plotting the test set results
  plt.scatter(y_test, y_pred)
  plt.xlabel('True values')
  plt.ylabel('Predictions')
  plt.title('True values vs Predictions (Neural Network)')
  plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], c='r')
  plt.legend(['Test values', 'Perfect fit'])
  plt.show()
  
  # Plot residuals
  residuals = y_pred - y_test
  plt.scatter(y_pred, residuals)
  plt.hlines(y=0, xmin=y_test.min(), xmax=y_test.max(), colors='r')
  plt.title('Residual plot (Neural Network)')
  plt.xlabel('Predicted values')
  plt.ylabel('Residuals')
  plt.legend(['Residuals', 'Perfect fit'])
  plt.show()
  return None

In [ ]:
neural_network(X_train, y_train, X_test, y_test)

neural_network(X_train, y_train, X_test, y_test)